# Modelo de Recomendación de películas

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

In [249]:
df = pd.read_csv('../clean_data/movies_model.csv')
# x = 0
# for i in range(len(df)):
#     print(i)
#     if len(df.word_bag[i]) > x:
#         x = len(df.word_bag[i])
# print(x)

In [57]:
df

,id_peli,title,word_bag
0,862,Toy Story,ToyStoryCollection Animation Comedy Family Joh...
1,8844,Jumanji,Family Adventure Fantasy JoeJohnston JoeJohnston
2,15602,Grumpier Old Men,GrumpyOldMenCollection Comedy Romance HowardDe...
3,31357,Waiting To Exhale,Comedy Romance Drama ForestWhitaker ForestWhi...
4,11862,Father Of The Bride Part Ii,Comedy CharlesShyer CharlesShyer
...,...,...,...
13131,432789,The Incredible Jessica James,Comedy Romance JimStrouse JimStrouse
13132,434873,It Stains The Sands Red,Comedy Drama Horror ColinMinihan ColinMinihan
13133,18098,Arabian Nights,Family Fantasy Drama SteveBarron SteveBarron
13134,455661,In A Heartbeat,Animation Comedy Family Romance BethDavid Est...


In [250]:
#stop_words attribute will skip all the helper words that occur english language eg: in, the , they etc.
# del(vector)
# del(cv_fvector)
# del(similarity_array)
cv_fvector = CountVectorizer(stop_words='english')
vector = cv_fvector.fit_transform(df['word_bag']).toarray()
vector.shape

(15076, 18791)

In [251]:
similarity_array = cosine_similarity(vector,vector)
similarity_array
# similarity_array = linear_kernel(vector,vector)

array([[1.        , 0.1118034 , 0.1       , ..., 0.10540926, 0.26311741,
        0.11952286],
       [0.1118034 , 1.        , 0.        , ..., 0.23570226, 0.09805807,
        0.        ],
       [0.1       , 0.        , 1.        , ..., 0.        , 0.1754116 ,
        0.11952286],
       ...,
       [0.10540926, 0.23570226, 0.        , ..., 1.        , 0.09245003,
        0.        ],
       [0.26311741, 0.09805807, 0.1754116 , ..., 0.09245003, 1.        ,
        0.10482848],
       [0.11952286, 0.        , 0.11952286, ..., 0.        , 0.10482848,
        1.        ]])

In [64]:
sorted(list(enumerate(similarity_array[df[df.id_peli == 862].index.item()])),reverse=True,key = lambda x: x[1])[1:6]

[(1235, 0.9999999999999998),
 (998, 0.8749999999999998),
 (3693, 0.6864064729836441),
 (6178, 0.6864064729836441),
 (8071, 0.6708203932499369)]

In [87]:
# Recomendaciones con cosine_similarity
df = pd.read_csv('../clean_data/movies_model.csv')
# Recomendación por medio de id_peli.
recommended_five = sorted(list(enumerate(similarity_array[df[df.id_peli == 862].index.item()])),reverse=True,key = lambda x: x[1])[1:6]
# Recomendación por medio de title.
#recommended_five = sorted(list(enumerate(similarity_array[df.index[df.title.isin(['Toy Story'])].item()])),reverse=True,key = lambda x: x[1])[1:6]
lista = []
for i in recommended_five:
    lista.append(i[0])
print(f'La película que ingresaste es: {df[df.id_peli == 862].title.item()}. Las recomendaciones son: \n')
for x in df.title.loc[lista]:
    print(x)

La película que ingresaste es: Toy Story. Las recomendaciones son: 

Toy Story 2
A Bug'S Life
Cars
Cars 2
Mater And The Ghostlight


In [231]:
sorted(list(enumerate(similarity_array[df[df.id_peli == 218].index.item()])),reverse=True,key = lambda x: x[1])[:6]

[(402, 1.0),
 (642, 1.0),
 (585, 0.8888888888888888),
 (620, 0.8432740427115678),
 (265, 0.816496580927726),
 (6568, 0.7453559924999298)]

In [234]:
df.loc[[402,642]]

,id_peli,title,word_bag
402,280,Terminator 2: Judgment Day,TheTerminatorCollection Action Thriller Scienc...
642,218,The Terminator,TheTerminatorCollection Action Thriller Scienc...


In [252]:
dfx = pd.DataFrame(columns=['id_peli','idx_recommend'])
for i in df.id_peli:
    recommended_five = sorted(list(enumerate(similarity_array[df[df.id_peli == i].index.item()])),reverse=True,key = lambda x: x[1])[1:6]
    lista = []
    for j in recommended_five:
        lista.append([i,j[0]])
    dfx = pd.concat([dfx,pd.DataFrame(lista,columns=['id_peli','idx_recommend'])])    

In [253]:
dfx = dfx.merge(df).drop('word_bag',axis=1)[['id_peli', 'title','idx_recommend']]
dfx.to_csv('../clean_data/recommended_movies.csv')
dfx[dfx.title == 'The Terminator']

,id_peli,title,idx_recommend
3210,218,The Terminator,402
3211,218,The Terminator,585
3212,218,The Terminator,620
3213,218,The Terminator,6568
3214,218,The Terminator,265


In [245]:
def recomendacion(pelicula:[str,int]):
    if type(pelicula) == str:
        pelicula = pelicula.title()
        lista = list(dfx[dfx.title == pelicula].idx_recommend)
    elif type(pelicula) == int:
        lista = list(dfx[dfx.id_peli == pelicula].idx_recommend)
        pelicula = dfx.title[dfx.id_peli == pelicula][:1].item()
    
    recomendacion = df.loc[lista][['id_peli', 'title']]
    recomendacion.columns = ['id_peli','recomendaciones']
    
    if not recomendacion.empty:
        return {'Ingresaste la película: ' : f'{pelicula}', 
                'Las recomendaciones son: ' : f'1.- [{recomendacion.recomendaciones[0:1].item()}]; 2.- [{recomendacion.recomendaciones[1:2].item()}]; 3.- [{recomendacion.recomendaciones[2:3].item()}]; 4.- [{recomendacion.recomendaciones[3:4].item()}]; 5.- [{recomendacion.recomendaciones[4:5].item()}]'}
    else: 
        return {'No se encontró la película: ' : f'{pelicula}'}

In [256]:
# Sucedió que en la recomendación de películas, la película misma que es la que debio dar 1, estuviera ordenada en segundo lugar, ya que había otra película diferente con la diferencia = 1
recomendacion('the terminator')

C:\Users\tinma\AppData\Local\Temp\ipykernel_23348\2823260797.py:14: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  'Las recomendaciones son: ' : f'1.- [{recomendacion.recomendaciones[0:1].item()}]; 2.- [{recomendacion.recomendaciones[1:2].item()}]; 3.- [{recomendacion.recomendaciones[2:3].item()}]; 4.- [{recomendacion.recomendaciones[3:4].item()}]; 5.- [{recomendacion.recomendaciones[4:5].item()}]'}


{'Ingresaste la película: ': 'The Terminator',
 'Las recomendaciones son: ': '1.- [Terminator 2: Judgment Day]; 2.- [The Abyss]; 3.- [Aliens]; 4.- [Terminator Salvation]; 5.- [True Lies]'}

In [255]:
df.loc[402]

id_peli                                                   280
title                              Terminator 2: Judgment Day
word_bag    Terminator 2: Judgment Day TheTerminatorCollec...
Name: 402, dtype: object

In [216]:
dfy = pd.read_csv('../Data/movies_dataset.csv')
dfy[dfy]

C:\Users\tinma\AppData\Local\Temp\ipykernel_23348\4192713340.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  dfy = pd.read_csv('../Data/movies_dataset.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
1195,False,"{'id': 528, 'name': 'The Terminator Collection...",6400000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",http://www.mgm.com/#/our-titles/1970/The-Termi...,218,tt0088247,en,The Terminator,"In the post-apocalyptic future, reigning tyran...",...,1984-10-26,78371200.0,108.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Your future is in his hands.,The Terminator,False,7.4,4208.0
